In [124]:
#pip install --upgrade pip
# #!pip install transformers
#!pip install gensim
#!pip install spacy #rather use Terminal with:
#!pip install -U spacy
#!pip install nltk

In [125]:
# Step 1 importing packages
import pandas as pd
import re
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
from transformers import BertTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [107]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [108]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v2.csv'
df = pd.read_csv(url)
df.head()


,data,player,language
0,{'content': 'Manchester\xa0United\xa0imploded ...,Mitchel Bakker,en
1,{'content': 'Slick Sevilla punish slapdash Man...,Mitchel Bakker,en
2,{'content': 'Manchester United imploded in Sev...,Mitchel Bakker,en
3,{'content': 'Bayer Leverkusen has reached the ...,Mitchel Bakker,en
4,{'content': 'Leverkusen\'s players celebrate a...,Mitchel Bakker,en


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   data      445 non-null    object
 1   player    445 non-null    object
 2   language  445 non-null    object
dtypes: object(3)
memory usage: 10.6+ KB


In [110]:
df['player'].value_counts()

Moussa Diaby         130
Jeremie Frimpong     112
Jonathan Tah          68
Exequiel Palacios     52
Mitchel Bakker        42
Mykhaylo Mudryk       34
Piero Hincapie         7
Name: player, dtype: int64

In [111]:
df['language'].value_counts()
#for more about that, pls see "Language_check.ipynb"

de    309
en    104
es     32
Name: language, dtype: int64

## Make data in small letters

In [112]:
data_lower = df.copy()

In [113]:
data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()
data_lower.head()

,data,player,language
0,{'content': 'manchester\xa0united\xa0imploded ...,mitchel bakker,en
1,{'content': 'slick sevilla punish slapdash man...,mitchel bakker,en
2,{'content': 'manchester united imploded in sev...,mitchel bakker,en
3,{'content': 'bayer leverkusen has reached the ...,mitchel bakker,en
4,{'content': 'leverkusen\'s players celebrate a...,mitchel bakker,en


## Clean data from patterns

In [114]:
'''
To Dos:
- cut in the beginning: {'content': '
- cut in the end: \nAdvertisement\nAdvertisement\nAdvertisement'}
'''
data_wo_pattern = data_lower.copy()

In [115]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"\\nAdvertisement\\nAdvertisement\\nAdvertisement'\}$", "", str(x)))
print(data_wo_pattern.iloc[0, 0])


manchester\xa0united\xa0imploded in seville in the europa league on thursday while juventus completed a good evening for the club by holding on away to sporting and advancing to the semi-finals.in the quarter-final second legs,\xa0united\xa0succumbed 3-0 at sevilla to go out 5-2.juventus hung on to draw 1-1 with sporting in lisbon to advance 2-1 on aggregate while bayer leverkusen won 4-1 in belgium to eliminate union saint-gilloise 5-2.jose mourinho\'s roma beat feyenoord 4-1 after extra time to advance 4-2.united, who led by two goals with six minutes left in the first leg before conceding a pair of late own goals, completed the collapse in spain.sponsored by vuuklegoalkeeper david de gea had a disastrous night as lacklustre\xa0united\xa0were dispatched by the six-time competition winners.after eight minutes de gea rolled a pass to an isolated harry maguire, scorer of one of the first-leg own goals. this time, ambushed by three opponents, he gave the ball away and youssef en-nesyri a

## Clean data from non-textual data

In [116]:
'''
To Dos:
- strip numeric
- strip punctuation
- strip multiple whitespaces
'''
data_stripped = data_wo_pattern.copy()

In [117]:
# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_stripped['data'] = data_stripped['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_stripped['data'] = data_stripped['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_stripped['data'] = data_stripped['data'].apply(strip_multiple_whitespaces)

data_stripped.head()

,data,player,language
0,manchester xaunited xaimploded in seville in t...,mitchel bakker,en
1,slick sevilla punish slapdash man united to ea...,mitchel bakker,en
2,manchester united imploded in seville in the e...,mitchel bakker,en
3,bayer leverkusen has reached the semifinal of ...,mitchel bakker,en
4,leverkusen s players celebrate after their tea...,mitchel bakker,en


## Clean data from stopwords

In [118]:
'''QUESTION for us: shall we decide to keep stop words? Or keep some?'''

#nltk
stop_words = nltk.corpus.stopwords.words('english') 
print(sorted(stop_words),'\n', len(stop_words))

# gensim
print(sorted(STOPWORDS),'\n', len(STOPWORDS))

# Create a list of stop words to remove 
stop_words_to_remove = ["aren't",'isn', "isn't",'mightn', "mightn't",'mustn', "mustn't",'needn', "needn't",'no', 'nor', 'not',"shan't",'shouldn', "shouldn't",'wasn', "wasn't","weren't",'wouldn', "wouldn't"] 
'''what are we missing?'''

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  stop_words.remove(word)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some',

In [119]:
'''depending on what we decided?'''

'''data_rm = data_stripped.copy()'''

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=stop_words)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_wostop['data'] = data_rm['data'].apply(remove_stopwords_from_text)

NameError: name 'data_rm' is not defined

## Clean data from too short words 

In [126]:
'''Anpassung nötig'''
data_noshortwords = data_stripped.copy() #or
'''data_noshortwords = data_wostop.copy()
'''

# strip_short deletes words smaller 3
data_noshortwords['data'] = data_noshortwords['data'].apply(strip_short)

data_noshortwords['data']

0      manchester xaunited xaimploded seville the eur...
1      slick sevilla punish slapdash man united ease ...
2      manchester united imploded seville the europa ...
3      bayer leverkusen has reached the semifinal eur...
4      leverkusen players celebrate after their team ...
                             ...                        
440    content nick emms for mailonline npublished bs...
441    arsenal have released statement criticise the ...
442    content will griffee for mailonline npublished...
443    untold arsenal supporting the club the manager...
444    content editions nen football chelsea nowners ...
Name: data, Length: 445, dtype: object

## Comparing the results of stemming, lemmatization and tokenization

In [127]:
# stem_text transforms the data into root form w/o using a dictionary by certain rules -> Return lowercase and (porter-)stemmed version of string text.
data_stem = data_noshortwords['data'].apply(stem_text)
data_stem = data_stem.tolist()
data_stem

['manchest xaunit xaimplod sevil the europa leagu thursdai while juventu complet good even for the club hold awai sport and advanc the semi final the quarter final second leg xaunit xasuccumb sevilla out juventu hung draw with sport lisbon advanc aggreg while bayer leverkusen won belgium elimin union saint gillois jose mourinho roma beat feyenoord after extra time advanc unit who led two goal with six minut left the first leg befor conced pair late own goal complet the collaps spain sponsor vuuklegoalkeep david gea had disastr night lacklustr xaunit xawer dispatch the six time competit winner after eight minut gea roll pass isol harri maguir scorer on the first leg own goal thi time ambush three oppon gave the ball awai and youssef nesyri accept the present sevilla domin and could have score again befor centr back loic bade head onto hi own shoulder and loop the ball over gea the minut the minut gea charg out but slice drop ball fell nesyri who curl into the empti net complet the victo

In [129]:
data_lem = data_noshortwords.copy()

#en_model = en_core_web_sm.load()


def apply_lemmatization(text):
    # Parse the text with spaCy
    doc = nlp(text)
    # Extract the lemmas for each token
    lemmas = [token.lemma_ for token in doc]
    # Join the lemmas with a space separator
    return " ".join(lemmas)

# Apply the lemmatization function to the 'text' column using the apply method
data_lem['lemmatized_text'] = data_lem['data'].apply(apply_lemmatization)
data_lem

,data,player,language,lemmatized_text
0,manchester xaunited xaimploded seville the eur...,mitchel bakker,en,manchester xaunite xaimplode seville the europ...
1,slick sevilla punish slapdash man united ease ...,mitchel bakker,en,slick sevilla punish slapdash man unite ease i...
2,manchester united imploded seville the europa ...,mitchel bakker,en,manchester united imploded seville the europa ...
3,bayer leverkusen has reached the semifinal eur...,mitchel bakker,en,bayer leverkusen have reach the semifinal euro...
4,leverkusen players celebrate after their team ...,mitchel bakker,en,leverkusen player celebrate after their team s...
...,...,...,...,...
440,content nick emms for mailonline npublished bs...,jeremie frimpong,en,content nick emms for mailonline npublishe bst...
441,arsenal have released statement criticise the ...,mykhaylo mudryk,en,arsenal have release statement criticise the b...
442,content will griffee for mailonline npublished...,mykhaylo mudryk,en,content will griffee for mailonline npublishe ...
443,untold arsenal supporting the club the manager...,mykhaylo mudryk,en,untold arsenal support the club the manager an...


In [130]:
data_bert = data_noshortwords.copy() 

# define bert tokenizer
bert_uncased = BertTokenizer.from_pretrained('bert-base-uncased')

# print out tokens
print(list(bert_uncased.vocab.keys())[1997:2100])

C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joana\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


['of', 'and', 'in', 'to', 'was', 'he', 'is', 'as', 'for', 'on', 'with', 'that', 'it', 'his', 'by', 'at', 'from', 'her', '##s', 'she', 'you', 'had', 'an', 'were', 'but', 'be', 'this', 'are', 'not', 'my', 'they', 'one', 'which', 'or', 'have', 'him', 'me', 'first', 'all', 'also', 'their', 'has', 'up', 'who', 'out', 'been', 'when', 'after', 'there', 'into', 'new', 'two', 'its', '##a', 'time', 'would', 'no', 'what', 'about', 'said', 'we', 'over', 'then', 'other', 'so', 'more', '##e', 'can', 'if', 'like', 'back', 'them', 'only', 'some', 'could', '##i', 'where', 'just', '##ing', 'during', 'before', '##n', 'do', '##o', 'made', 'school', 'through', 'than', 'now', 'years', 'most', 'world', 'may', 'between', 'down', 'well', 'three', '##d', 'year', 'while', 'will', '##ed', '##r']


In [131]:
data_bert = data_noshortwords['data'] 

# convert into list 
data_bert = data_bert.tolist()

# run tokenizer over all list objects
data_BERT = [bert_uncased.tokenize(text) for text in data_bert]
data_BERT[0]

['manchester',
 'x',
 '##au',
 '##ni',
 '##ted',
 'x',
 '##ai',
 '##mp',
 '##lo',
 '##ded',
 'seville',
 'the',
 'europa',
 'league',
 'thursday',
 'while',
 'juventus',
 'completed',
 'good',
 'evening',
 'for',
 'the',
 'club',
 'holding',
 'away',
 'sporting',
 'and',
 'advancing',
 'the',
 'semi',
 'finals',
 'the',
 'quarter',
 'final',
 'second',
 'legs',
 'x',
 '##au',
 '##ni',
 '##ted',
 'x',
 '##as',
 '##uc',
 '##cum',
 '##bed',
 'sevilla',
 'out',
 'juventus',
 'hung',
 'draw',
 'with',
 'sporting',
 'lisbon',
 'advance',
 'aggregate',
 'while',
 'bayer',
 'lever',
 '##kus',
 '##en',
 'won',
 'belgium',
 'eliminate',
 'union',
 'saint',
 'gill',
 '##oise',
 'jose',
 'mo',
 '##uri',
 '##nh',
 '##o',
 'roma',
 'beat',
 'fey',
 '##eno',
 '##ord',
 'after',
 'extra',
 'time',
 'advance',
 'united',
 'who',
 'led',
 'two',
 'goals',
 'with',
 'six',
 'minutes',
 'left',
 'the',
 'first',
 'leg',
 'before',
 'con',
 '##ced',
 '##ing',
 'pair',
 'late',
 'own',
 'goals',
 'completed